# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [1]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

In [4]:
from osc_ingest_trino import *

trino_bucket = attach_s3_bucket("S3_DEV")

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [6]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sec_dera'

In [7]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epa_frs',)
('epa_ghgrp',)
('epacems',)
('epacems_y95_al',)
('essd',)
('ghgrp_demo',)
('gleif',)
('gleif_mdt',)
('iceberg_demo',)
('information_schema',)
('ingest_schema',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('physical_risk_project',)
('pudl',)
('rmi_20210929',)
('rmi_20211120',)
('rmi_utility_transition_hub',)
('sec_dera',)
('sfi_geoasset',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('us_census',)
('wri',)
('wri_demo',)
('wri_dev',)
('wri_gppd',)
('wri_gppd_md',)
('wri_new',)
('wri_test',)


Enter the Pandas!

In [8]:
import pandas as pd
import io

Drop previous tables and schema to start with a fresh slate

In [9]:
for view in [ 'assets_by_adsh_ddate', 'assets_by_lei',
               'cash_by_adsh_ddate', 'cash_by_lei',
               'debt_by_adsh_ddate', 'debt_by_lei',
               'financials_by_lei',
               'float_by_adsh_ddate', 'float_by_lei',
               'fy_revenue_by_lei']:
    sql = f"""
drop view if exists {ingest_catalog}.{ingest_schema}.{view}
"""
    # print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())

for ingest_table in [ 'sub', 'num', 'tag', 'ticker',
                      't_a', 't_c', 't_d', 't_f', 't_r']:
    sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
    # print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())

qres = engine.execute(f"show tables in {ingest_schema}")
print(qres.fetchall())

sql = f"""
drop schema if exists {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[]

drop schema if exists osc_datacommons_dev.sec_dera

[(True,)]


In [10]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


create schema osc_datacommons_dev.sec_dera

[(True,)]


Load `ticker` file (updated sporadically from https://www.sec.gov/include/ticker.txt)

In [11]:
ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'SEC-DERA/ticker.txt')
ticker_file.download_file(f'/tmp/dera-ticker.txt')
ticker_df = pd.read_csv(f'/tmp/dera-ticker.txt', names=['tname', 'cik'], header=None, sep='\t', dtype={'tname':'string','cik':'int64'}, engine='c')
ticker_dict = dict(zip(ticker_df.cik, ticker_df.tname))

In [12]:
ticker_df

,tname,cik
0,aapl,320193
1,msft,789019
2,goog,1652044
3,amzn,1018724
4,tsla,1318605
...,...,...
12416,afacu,1849489
12417,afaqw,1841661
12418,aesc,874761
12419,aesew,1708341


In [13]:
buf = io.BytesIO()
ticker_df.to_parquet(path=buf)
buf.seek(0)
trino_bucket.upload_fileobj(Fileobj=buf,
                      Key=f'trino/{ingest_schema}/ticker/data.parquet')

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.ticker;
create table {ingest_catalog}.{ingest_schema}.ticker(
    cik bigint,
    tname varchar
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/ticker/'
);
select count (*) from {ingest_catalog}.{ingest_schema}.ticker;
select * from {ingest_catalog}.{ingest_schema}.ticker limit 10
"""
for sql_stmt in sql.split(';'):
    print(sql_stmt)
    qres = engine.execute(sql_stmt)
    print(qres.fetchall())


drop table if exists osc_datacommons_dev.sec_dera.ticker
[(True,)]

create table osc_datacommons_dev.sec_dera.ticker(
    cik bigint,
    tname varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/sec_dera/ticker/'
)
[(True,)]

select count (*) from osc_datacommons_dev.sec_dera.ticker
[(12421,)]

select * from osc_datacommons_dev.sec_dera.ticker limit 10

[(320193, 'aapl'), (789019, 'msft'), (1652044, 'goog'), (1018724, 'amzn'), (1318605, 'tsla'), (1326801, 'fb'), (40545, 'ge'), (1067983, 'brk-a'), (1046179, 'tsm'), (1293451, 'tcehy')]


Prepare GLEIF matching data

In [14]:
gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = dict(zip(gleif_df.name, gleif_df.LEI))

Load the SUB, NUM, and TAG tables into Trino

In [15]:
import re
import uuid

# Add a unique identifier to the data set
uid = str(uuid.uuid4())

dera_regex = re.compile(r' ?/.*$')

def ingest_dera_table(zf, fy_qtr, tbl, ftimestamp):
    """From a local file ZF, read data for the period FY_QTR for the DERA table TBL.
    Return the Dataframe created so that when it is time to create the actual Trino table
    we know what the shape of the data should look like.  The returned DF has all the data
    of the specific ingestion, not all the data of all the ingestions of data for TBL."""
    df = pd.read_csv(zf, header=0, sep='\t', dtype='string', keep_default_na=False, nrows = None, engine='c')
    df['srcdir'] = fy_qtr
    df.srcdir = df.srcdir.astype('string')
    df['uuid'] = uid
    df.uuid = df.uuid.astype('string')
    
    # df = df.convert_dtypes (infer_objects=False, convert_string=True, convert_integer=False, convert_boolean=False, convert_floating=False)
    # Print the output
    # print(df.dtypes)
    
    if tbl=='sub':
        df.name = df.name.map(lambda x: re.sub(dera_regex, '', x))
        df.name = df.name.astype('string')
        df['LEI'] = df.name.map(gleif_dict)
        df.LEI = df.LEI.astype('string')
        df.cik = df.cik.astype('int32')
        df.loc[df.sic=='', 'sic'] = pd.NA
        df.sic = df.sic.astype('Int16')
        df.loc[df.ein=='', 'ein'] = pd.NA
        df.ein = df.ein.astype('Int64')
        df.wksi = df.wksi.astype('bool')
        # df.wksi = df.wksi.astype('int32')
        df.period = pd.to_datetime(df.period, format='%Y%m%d', utc=True, errors='coerce')
        df.fy = pd.to_datetime(df.fy, format='%Y', utc=True, errors='coerce')
        df.filed = pd.to_datetime(df.filed, format='%Y%m%d', utc=True)
        df.accepted = pd.to_datetime(df.accepted, format='%Y-%m-%d %H:%M:%S', utc=True)
        df.prevrpt = df.prevrpt.astype('bool')
        df.detail = df.detail.astype('bool')
        df.nciks = df.nciks.astype('int16')
        
        cols = df.columns.tolist()
        # Move LEI to a more friendly location in the column order
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tbl=='num':
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        df.loc[df.ddate=='30210630', 'ddate'] = '20210630'
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d', utc=True)
        df.qtrs = df.qtrs.astype('int16')
        df.loc[df.coreg=='', 'coreg'] = pd.NA
        df.loc[df.value=='', 'value'] = pd.NA
        df.value = df.value.astype('Float64')
        df.loc[df.footnote=='', 'footnote'] = pd.NA
    elif tbl=='tag':
        df.custom = df.custom.astype('bool')
        df.abstract = df.abstract.astype('bool')
        df.loc[df.crdr=='', 'crdr'] = pd.NA
        df.loc[df.tlabel=='', 'tlabel'] = pd.NA
        df.loc[df.doc=='', 'doc'] = pd.NA
    # print(df.dtypes)
    # display(df.head())

    buf = io.BytesIO()
    df.to_parquet(path=buf)
    buf.seek(0)
    trino_bucket.upload_fileobj(Fileobj=buf,
                                Key=f'trino/{ingest_schema}/{tbl}/{fy_qtr}.parquet')
    return df

In [16]:
import io
import zipfile
import datetime

objects=source_bucket.objects.filter(Prefix='SEC-DERA/20')

dera_tables = ['sub', 'num', 'tag']
dera_df = {}

for obj in objects:
    if obj.key.endswith('.zip'):
        zipfile_src = s3_source.Object(os.environ['S3_LANDING_BUCKET'],obj.key)
        tmpname = obj.key.split('/')[-1]
        zipfile_src.download_file(f'/tmp/{tmpname}')
        zipfile_obj = zipfile.ZipFile(f'/tmp/{tmpname}', mode='r')
        fy_qtr = tmpname.split('.')[0]
        for zipinfo in zipfile_obj.infolist():
            fname = zipinfo.filename
            if fname[3:] != '.txt':
                continue
            tbl = fname[:3]
            if tbl not in dera_tables:
                continue
            ftimestamp = datetime.datetime(*zipinfo.date_time)
            print(f'{fy_qtr} - {tbl}')
            with zipfile_obj.open(fname) as zf:
                # This fills a directory with parquet files
                dera_df[tbl] = ingest_dera_table (zf, fy_qtr, tbl, ftimestamp)
        zipfile_obj.close()

# Once we have all our parquet files in place, load up the tables with their directory contents
for tbl in dera_tables:
    if tbl not in dera_df:
        error(f'{tbl} data not found')
    table_check = engine.execute(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tbl}')
    for row in table_check.fetchall():
        print(row)

    columnschema = create_table_schema_pairs(dera_df[tbl], typemap={'int16':'smallint', 'Int16':'smallint'})
    tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{tbl} (
{columnschema}
) with (
format = 'parquet',
external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{tbl}/'
)
"""
    print(tabledef)

    table_create = engine.execute(tabledef)
    for row in table_create.fetchall():
        print(row)

    dataset_query = (f'SELECT * FROM {ingest_catalog}.{ingest_schema}.{tbl} limit 10')
    print(dataset_query)
    dataset = engine.execute(dataset_query)
    for row in dataset.fetchall():
        print(row)

2020q1 - sub
2020q1 - tag
2020q1 - num
2020q2 - sub
2020q2 - tag
2020q2 - num
2020q3 - sub
2020q3 - tag
2020q3 - num
2020q4 - sub
2020q4 - tag
2020q4 - num
2021q1 - sub
2021q1 - tag
2021q1 - num
2021q2 - sub
2021q2 - tag
2021q2 - num
2021q3 - sub
2021q3 - tag
2021q3 - num
(True,)

create table if not exists osc_datacommons_dev.sec_dera.sub (
    adsh varchar,
    cik integer,
    name varchar,
    LEI varchar,
    sic smallint,
    countryba varchar,
    stprba varchar,
    cityba varchar,
    zipba varchar,
    bas1 varchar,
    bas2 varchar,
    baph varchar,
    countryma varchar,
    stprma varchar,
    cityma varchar,
    zipma varchar,
    mas1 varchar,
    mas2 varchar,
    countryinc varchar,
    stprinc varchar,
    ein bigint,
    former varchar,
    changed varchar,
    afs varchar,
    wksi boolean,
    fye varchar,
    form varchar,
    period timestamp,
    fy timestamp,
    fp varchar,
    filed timestamp,
    accepted timestamp,
    prevrpt boolean,
    detail boolean,


In [17]:
# Borrow metadata code from DERA-iceberg if/when we need it

In [18]:
tablenames = ['sub', 'num', 'tag', 'ticker']
l = []
for tbl in tablenames:
    qres = engine.execute(f'select count (*) from {ingest_catalog}.{ingest_schema}.{tbl}')
    l.append(qres.fetchall()[0])
print(list(zip(tablenames, l)))

[('sub', (119563,)), ('num', (18053842,)), ('tag', (705825,)), ('ticker', (12421,))]


In [26]:
qres = engine.execute("""
select count (*), tag, uom, min(value) as minval, max(value) as maxval from sec_dera.num where tag like '%ExchangeRate%' and uom like '%/%' group by tag, uom order by count (*) desc
""")
display(qres.fetchall())

[(56, 'ForeignCurrencyExchangeRateTranslation1', 'USD/CNY', 0.1415, 6.9762),
 (44, 'ForeignCurrencyExchangeRateTranslation1', 'CNY/USD', 0.0, 7.1383),
 (43, 'AverageForeignExchangeRate', 'USD/EUR', 0.8455, 1.2271),
 (43, 'ForeignCurrencyExchangeRateTranslation1', 'USD/GBP', 0.71, 1.4),
 (42, 'ClosingForeignExchangeRate', 'USD/GBP', 0.72, 1.3836),
 (38, 'ClosingForeignExchangeRate', 'USD/EUR', 0.8141, 1.2271),
 (35, 'AverageForeignExchangeRate', 'USD/GBP', 0.72, 1.3913),
 (33, 'ForeignCurrencyExchangeRateTranslation1', 'USD/EUR', 1.1, 1.23),
 (27, 'AverageForeignExchangeRate', 'EUR/USD', 1.1191, 1.2),
 (26, 'ClosingForeignExchangeRate', 'MXN/USD', 18.8452, 24.623),
 (24, 'ClosingForeignExchangeRate', 'EUR/USD', 1.1029, 1.2321),
 (22, 'AverageForeignExchangeRate', 'BRL/EUR', 3.605, 6.4902),
 (20, 'AverageForeignExchangeRate', 'GBP/EUR', 0.874, 0.89),
 (19, 'ForeignCurrencyExchangeRateTranslation1', 'USD/CAD', 0.71, 1.3639),
 (19, 'ClosingForeignExchangeRate', 'BRL/EUR', 3.9693, 6.3779),


In [56]:
qres = engine.execute("""
select count (*), tag from sec_dera.num where tag like '%ExchangeRate%' and uom like '%/%' group by tag order by count (*) desc
""")
l = qres.fetchall()
display(l)

[(552, 'ClosingForeignExchangeRate'),
 (500, 'AverageForeignExchangeRate'),
 (300, 'ForeignCurrencyExchangeRateTranslation1'),
 (44, 'AverageForeignCurrencyExchangeRateTranslation'),
 (15, 'ForeignCurrencyExchangeRateRemeasurement1'),
 (15, 'ForeignCurrencyTranslationWeightedAverageExchangeRate'),
 (11, 'DerivativeAverageForwardExchangeRate1'),
 (10, 'WeightedAverageExchangeRate'),
 (8, 'ForeignCurrencyAverageExchangeRateTranslation'),
 (6, 'AnnualAverageForeignCurrencyExchangeRateTranslation'),
 (6, 'CeilingTestsBenchmarkExchangeRate'),
 (6, 'AssetImpairmentRecoveryAssumptionsLongTermAverageExchangeRate'),
 (6, 'ForeignCurrencyExchangeRateTranslationAverage'),
 (6, 'AverageForeignCurrencyExchangeRate'),
 (4, 'IncreaseDecreaseOfAverageForeignCurrencyExchangeRateTranslation'),
 (4, 'ExchangeRatePerDollar'),
 (3, 'AverageForeignExchangeRateAssumption'),
 (3, 'ForeignCurrencyAverageExchangeRate'),
 (3, 'ForeignCurrencyExchangeRateTranslation2'),
 (3, 'NationalBankForeignExchangeRate'),
 (

In [74]:
qres = engine.execute("""
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, N.uom, N.value as assets, coalesce(FX_1.value,FX_2.value,FX_3.value) * N.value as assets_usd, coalesce(FX_1.tag,FX_2.tag,FX_3.tag)
from sec_dera.sub as S
     join sec_dera.assets_xyz_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.a_ddate=N.ddate and N.coreg is NULL and N.qtrs=0 and N.tag='Assets'
     left join sec_dera.num as FX_1 on N.adsh=FX_1.adsh and N.ddate=FX_1.ddate and FX_1.tag='AverageForeignExchangeRate' and (FX_1.uom like (N.uom || '/USD') or FX_1.uom like ('USD/' || N.uom))
     left join sec_dera.num as FX_2 on N.adsh=FX_2.adsh and N.ddate=FX_2.ddate and FX_2.tag='ClosingForeignExchangeRate' and (FX_2.uom like (N.uom || '/USD') or FX_2.uom like ('USD/' || N.uom))
     left join sec_dera.num as FX_3 on N.adsh=FX_3.adsh and N.ddate=FX_3.ddate and FX_3.tag='ForeignCurrencyExchangeRateTranslation1' and (FX_3.uom like (N.uom || '/USD') or FX_3.uom like ('USD/' || N.uom))
where coalesce(FX_1.value,FX_2.value,FX_3.value) IS NOT NULL
""")
l = qres.fetchall()

In [75]:
len(l)

100

In [76]:
l[0:20]

[('0001104659-20-117718', 1549107, 'MANCHESTER UNITED PLC', '213800CNW6K7XWOTB808', 7900, '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 'GBP', 1383466000.0, 1747732597.8000002, 'AverageForeignExchangeRate'),
 ('0001104659-20-042364', 1002242, 'ENI SPA', 'BUCRF72VH5RBN7X3VL35', 1311, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', 'EUR', 123440000000.0, 138252800000.0, 'AverageForeignExchangeRate'),
 ('0001193125-20-110807', 910631, 'COCA COLA FEMSA SAB DE CV', '5493008KEVFYPMGTXO33', 2086, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', 'MXN', 257839000000.0, 4862843540000.0, 'ClosingForeignExchangeRate'),
 ('0001104659-20-049056', 1677250, 'ZTO EXPRESS (CAYMAN) INC.', '549300SCJPK3YZJTJR78', 4210, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', 'CNY', 45890502000.0, 6589876087.2, 'ForeignCurrencyExchangeRateTranslation1'),
 ('0001047469-20-002522', 1757097, 'CENTOGENE N.V.', '52990098DNIHN4D98U15', 8071, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', '

In [55]:
import pandas as pd

df = pd.read_sql("""
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, N.uom, N.value as assets, FX.value * N.value as assets_usd, FX.*
from sec_dera.sub as S
     join sec_dera.assets_xyz_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.a_ddate=N.ddate
     join sec_dera.num as FX on N.adsh=FX.adsh and N.ddate=FX.ddate
where N.coreg is NULL
      and N.qtrs=0
      and N.tag='Assets'
      and FX.tag like '%ExchangeRate%'
      and (FX.uom like (N.uom || '/USD') or FX.uom like ('USD/' || N.uom))
""", engine, index_col=None)
df

,adsh,cik,name,lei,sic,fy,ddate,uom,assets,assets_usd,...,tag,version,coreg,ddate,qtrs,uom,value,footnote,srcdir,uuid
0,0001628280-20-015058,1023514,HARMONY GOLD MINING CO LTD,378900986D05F0C29811,1040,2020-01-01 00:00:00.000,2020-06-30 00:00:00.000,ZAR,4.469200e+10,7.740654e+11,...,ClosingForeignExchangeRate,ifrs/2019,None,2020-06-30 00:00:00.000,0,ZAR/USD,17.3200,None,2020q4,4d138708-45bb-4a34-a647-a67cb7a86c8c
1,0001104659-20-139121,1696355,BRIGHT SCHOLAR EDUCATION HOLDINGS LTD,30030029PAIA7JHF7N33,8200,2020-01-01 00:00:00.000,2020-08-31 00:00:00.000,CNY,1.082331e+10,7.411153e+10,...,ForeignCurrencyExchangeRateTranslation1,us-gaap/2020,None,2020-08-31 00:00:00.000,0,USD/CNY,6.8474,None,2020q4,4d138708-45bb-4a34-a647-a67cb7a86c8c
2,0001104659-20-140691,1722380,ONESMART INTERNATIONAL EDUCATION GROUP LTD,549300M48VYZESQCQV62,8200,2020-01-01 00:00:00.000,2020-08-31 00:00:00.000,CNY,7.898391e+09,5.408344e+10,...,ForeignCurrencyExchangeRateTranslation1,us-gaap/2020,None,2020-08-31 00:00:00.000,0,USD/CNY,6.8474,None,2020q4,4d138708-45bb-4a34-a647-a67cb7a86c8c
3,0001047469-20-000960,1333141,FRESENIUS MEDICAL CARE AG & CO. KGAA,549300CP8NY40UP89Q40,8090,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,EUR,3.293474e+10,2.931850e+10,...,ClosingForeignExchangeRate,ifrs/2019,None,2019-12-31 00:00:00.000,0,USD/EUR,0.8902,None,2020q1,4d138708-45bb-4a34-a647-a67cb7a86c8c
4,0001047469-20-000960,1333141,FRESENIUS MEDICAL CARE AG & CO. KGAA,549300CP8NY40UP89Q40,8090,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,EUR,3.293474e+10,2.942060e+10,...,AverageForeignExchangeRate,ifrs/2019,None,2019-12-31 00:00:00.000,4,USD/EUR,0.8933,None,2020q1,4d138708-45bb-4a34-a647-a67cb7a86c8c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0001558370-20-005751,1582581,VOXELJET AG,None,3555,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,EUR,6.230500e+07,6.999344e+07,...,ClosingForeignExchangeRate,ifrs/2019,None,2019-12-31 00:00:00.000,0,USD/EUR,1.1234,None,2020q2,4d138708-45bb-4a34-a647-a67cb7a86c8c
135,0001558370-20-005751,1582581,VOXELJET AG,None,3555,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,EUR,6.230500e+07,6.975045e+07,...,AverageForeignExchangeRate,ifrs/2019,None,2019-12-31 00:00:00.000,4,USD/EUR,1.1195,None,2020q2,4d138708-45bb-4a34-a647-a67cb7a86c8c
136,0001104659-20-060191,1729173,UXIN LTD,549300OV7Z440PI54J04,7389,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,CNY,5.383096e+09,3.755355e+10,...,ForeignCurrencyExchangeRateTranslation1,us-gaap/2019,None,2019-12-31 00:00:00.000,0,CNY/USD,6.9762,None,2020q2,4d138708-45bb-4a34-a647-a67cb7a86c8c
137,0001104659-20-072395,1633441,SECOO HOLDING LTD,5493006YAK6ZHW3YPP89,5990,2019-01-01 00:00:00.000,2019-12-31 00:00:00.000,CNY,4.997196e+09,3.478948e+10,...,ForeignCurrencyExchangeRateTranslation1,us-gaap/2019,None,2019-12-31 00:00:00.000,0,CNY/USD,6.9618,None,2020q2,4d138708-45bb-4a34-a647-a67cb7a86c8c
